In [12]:
import pandas as pd
import os
import joblib
from sklearn.preprocessing import LabelEncoder

In [13]:
folder_path = 'cleaned_data'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")

Folder 'cleaned_data' already exists.


In [ ]:
#each csv file takes ~5 seconds to load, compared to ~1.8s on parquet

data_path='data/training_set_4/training_set/'
counter=0
dirs = os.listdir(data_path)
for csv_file in dirs:

    df=pd.read_csv(data_path+csv_file)

    df.to_parquet(folder_path+'/train'+str(counter)+'.parquet')
    counter+=1
    break

KeyboardInterrupt: 

In [15]:
columns_to_encode=['track_id_clean','session_id','skip_1','skip_2','skip_3','not_skipped','hist_user_behavior_is_shuffle','premium','context_type','hist_user_behavior_reason_start','hist_user_behavior_reason_end']

In [27]:
#run only the first time

label_encoders = {}
df=pd.DataFrame()
counter=0

data_path='cleaned_data/'
dirs = os.listdir(data_path)

for file in dirs:
    df_temp=pd.read_parquet(data_path+file)
    for column in columns_to_encode:
        label_encoder = LabelEncoder()
        df_temp[column] = label_encoder.fit_transform(df_temp[column])
        label_encoders[column] = label_encoder


    df_temp.to_parquet(folder_path+'/train'+str(counter)+'.parquet')
    counter+=1

joblib_file_path = 'label_encoders.joblib'
joblib.dump(label_encoders, joblib_file_path)




['label_encoders.joblib']

In [ ]:
#optimize later using cudff, do not run now

df=pd.DataFrame()
for file in dirs:
    df_temp=pd.read_parquet(data_path+file)
    df=pd.concat((df,df_temp))
    print(df.shape)


NameError: name 'pd' is not defined

In [41]:
#after cudff, do not run now
df.to_parquet(folder_path+'/train_compiled'+'.parquet')

In [44]:
les = joblib.load('label_encoders.joblib')

In [18]:
df_track_1=pd.read_csv('data/track_features/track_features/tf_000000000000.csv')
df_track_2=pd.read_csv('data/track_features/track_features/tf_000000000001.csv')

In [30]:
le=les['track_id_clean']
df_track_1['track_id']=le.fit_transform(df_track_1['track_id'])
df_track_2['track_id']=le.fit_transform(df_track_2['track_id'])


In [38]:
df_track_1.shape

(1853311, 30)

In [32]:
df_track=pd.concat((df_track_1,df_track_2))

In [43]:
label_encoder=LabelEncoder()
df_track['mode']=label_encoder.fit_transform(df_track['mode'])
label_encoders['mode']=label_encoder

{'track_id_clean': LabelEncoder(), 'session_id': LabelEncoder(), 'skip_1': LabelEncoder(), 'skip_2': LabelEncoder(), 'skip_3': LabelEncoder(), 'not_skipped': LabelEncoder(), 'hist_user_behavior_is_shuffle': LabelEncoder(), 'premium': LabelEncoder(), 'context_type': LabelEncoder(), 'hist_user_behavior_reason_start': LabelEncoder(), 'hist_user_behavior_reason_end': LabelEncoder(), 'mode': LabelEncoder()}


In [46]:
joblib.dump(label_encoders,joblib_file_path)

['label_encoders.joblib']

In [47]:
df_track.shape

(3706388, 30)

In [48]:
df_track.to_parquet('cleaned_data/track_features.parquet')